<a href="https://colab.research.google.com/github/danielbauer1979/FI830/blob/main/FI830_HW7_Sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interest Rate Derivatives

Assume the Vasicek model:

$$
dr_t=\alpha(\beta-r_t)+\gamma dW_t
$$

with (risk-neutral) parameters $\alpha=0.3,\ \beta=3.5\%,\ \gamma=0.02,$ and $r_0=1.5\%$ reflecting the relevant yield curve.

## (a) Analytical Price
The analytical price of a $T$-year zero coupon bond (paying \$ 1 at time $T$) is

$$
P(F,t,T) = e^{-r_t A(t,T)- D(t,T)}
$$

where
$$
A(t,T) = \frac{1-e^{-\alpha(T-t)}}{\alpha}
$$
and
$$
A(t,T) = \left(\frac{\gamma^2}{2\alpha^2}-\beta\right)\left(A(t,T)(T-t)\right)+\frac{\gamma^2}{4\alpha}A^2(t,T)
$$

In [1]:
import numpy as np
import scipy.stats as st

In [2]:
# Parameters
alpha = 0.3
beta = 0.048
gamma = 0.02
r0 = 0.015

In [3]:
def anal_Vasicek(t,T,r_0):
  A_t_T = (1-np.exp(-alpha*(T-t)))/alpha
  D_t_T = (gamma**2/(2*alpha**2)-beta)*(A_t_T-(T-t))+(gamma**2/(4*alpha))*A_t_T**2
  return np.exp(-r_0*A_t_T-D_t_T)

In [4]:
P_0_5 = anal_Vasicek(0,5,r0)
print("The analytical price of a 5-year zero coupon bond with face of $1 is $%.3f." % P_0_5)

The analytical price of a 5-year zero coupon bond with face of $1 is $0.859.


## (b) Monte Carlo Simulation
Let $h$ be the step size. An Euler discretization gives
$$
\begin{align*}
r_{t+1} &= r_t + \Delta r_t \\
&= r_t + \alpha(\beta-r_t)h+\gamma(W_{t+1}-W_t) \\
&= r_t + \alpha(\beta-r_t)h+\gamma\sqrt{h}Z_{t+1}
\end{align*}
$$
where $Z_{t+1}\sim N(0,1).$

Then $P(0,T)=\mathbb{E}^{\mathbb{Q}}[e^{-\int_0^T r_s ds}\vert \mathcal{F}_0]$ can be approximated via Monte Carlo simulation
$$
\hat{P}(0,T) = \frac{1}{N}\sum_{n=1}^N f(h,n)
$$
where $N$ is the number of simulations and 
$$
f(h,n) = F \times \text{exp}\left\{-\left(\frac{1}{2}r_0+\sum_{j=1}^{k-1}r_j+\frac{1}{2}r_k\right)h\right\}
$$
where $F$ is the face value of the $T$-year zero coupon bond.

In [5]:
def MCVesicek(h,T,N):
  bondP = [0] * N
  for n in range(N):
    k = int(T/h+1)
    r_t = [r0] * k
    Z = np.random.normal(0,1,k-1)
    for i in range(1,k):
      r_t[i] = r_t[i-1] + alpha*(beta-r_t[i-1])*h + gamma*np.sqrt(h)*Z[i-1]
    r_hat = (0.5*r_t[0] + sum(r_t[1:-1]) + 0.5*r_t[-1]) * h #Approximate integral by the trapezoidal rule, https://en.wikipedia.org/wiki/Trapezoidal_rule
    bondP[n] = np.exp(-r_hat)
  return np.average(bondP)

In [6]:
MC_P = MCVesicek(1/365,5,10000)
print("The Monte Carlo price of a 5-year zero coupon bond with face of $1 is $%.3f." % MC_P)

The Monte Carlo price of a 5-year zero coupon bond with face of $1 is $0.859.


## (c) Interest Rate Swap
Given level notional amount $Q,$ the swap rate of a non-deferred interest rate swap is
$$
R = \frac{1-P_{t_n}}{\sum_{i=1}^n P_{t_i}}
$$

In [7]:
P_t_i = [anal_Vasicek(0,i,r0) for i in range(1,6)]
swap_rate = (1-P_t_i[-1])/sum(P_t_i) 
print("The swap rate is %.2f%%.\n" % float(swap_rate*100))

The swap rate is 3.05%.



In [8]:
np.random.seed(101)
P_t_i_MC = [MCVesicek(1/365,i,1000) for i in range(1,6)]
swap_rate_MC = (1-P_t_i_MC[-1])/sum(P_t_i_MC) 
print("The swap rate is %.2f%%.\n" % float(swap_rate_MC*100))

The swap rate is 3.07%.



## (d) Call Option
The value (at time $t$) of a European Call option on a $S$-year zero-coupon bond with an option maturity of $T$ years and strike price of $K$ is
$$
e^{-\int_t^T r_s\,ds}\text{max}\{P(F,T,S)-K,0\}= P(1,t,T)(P(F,T,S)-K)^+
$$

In [9]:
# Parameters
t = 0
T = 4
S = 5
K = 970
F = 1000

N = 10000
h = 1/365

In [10]:
np.random.seed(42)
discCallPyo = [0] * N
for n in range(N):
  k = int(T/h+1)
  r_t = [r0] * k
  Z = np.random.normal(0,1,k-1)
  for i in range(1,k):
    r_t[i] = r_t[i-1] + alpha*(beta-r_t[i-1])*h + gamma*np.sqrt(h)*Z[i-1]
  r_hat = (0.5*r_t[0] + sum(r_t[1:-1]) + 0.5*r_t[-1]) * h #Approximate integral by the trapezoidal rule, https://en.wikipedia.org/wiki/Trapezoidal_rule
  discCallPyo[n] = np.exp(-r_hat) * max(1000*anal_Vasicek(T,S,r_t[-1])-K,0)
BondOption = np.average(discCallPyo)

In [11]:
print("The Bond option price is $%.3f." % BondOption)

The Bond option price is $4.386.
